In [1]:
import os
import numpy as np
from statistics import mean

import torch
import torchvision

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from scipy import stats

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

PROJECT_PATH = os.getenv('HOME') + '/aiffel/0_felton'
MODEL_PATH = os.path.join(PROJECT_PATH, 'weights')
DATA_PATH = os.path.join(PROJECT_PATH, 'data_ex3_refine')
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
REJECT_PATH = os.path.join(DATA_PATH, 'reject')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # 여기서 'cuda'가 출력되어야 GPU와 연결이 됩니다

cuda


In [ ]:
# train path image 확인 각폴더당 1개
for dirpath, dirnames, filenames in os.walk(TRAIN_PATH):
    for i, filename in enumerate(filenames):
        print(os.path.join(dirpath, filename)) # 파일이름을 출력합니다
        image = Image.open(os.path.join(dirpath, filename), 'r')
        print(f'size: ({image.width}, {image.height}, {image.getbands()})') # 이미지 정보를 출력합니다
        plt.imshow(image)
        plt.show()
        break # 폴더마다 1장만 출력합니다

In [3]:
# reject foler image 확인 5개
for dirpath, dirnames, filenames in os.walk(REJECT_PATH):
    for i, filename in enumerate(filenames):
        if i > 4:
            break
        print(os.path.join(dirpath, filename))
        image = Image.open(os.path.join(dirpath, filename), 'r')
        print(f'size: ({image.width}, {image.height}, {image.getbands()})')
        plt.imshow(image)
        plt.show()

In [4]:
# data 전처리 함수 만들기 resize 224, 224 -> resNet 사용 shape조절

# train data image 증강
# random H-flip, V-flip image 증강
# 색깔 바꾸기 colorjitter 
# center crop 사용안함

def create_dataloader(path, batch_size, istrain):
    nearest_mode = torchvision.transforms.InterpolationMode.NEAREST
    normalize = torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
    train_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
#         torchvision.transforms.CenterCrop((224,224)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ColorJitter(),
        torchvision.transforms.ToTensor(),
        normalize
    ])

    test_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
#         torchvision.transforms.CenterCrop((224,224)),
        torchvision.transforms.ToTensor(),
        normalize
    ])
    
    if istrain:
        data = torchvision.datasets.ImageFolder(path, transform=train_transformer)
        dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
        
    else:
        data = torchvision.datasets.ImageFolder(path, transform=test_transformer)
        dataloader = torch.utils.data.DataLoader(data, shuffle=False)

    return dataloader, data


In [5]:
# 위에서 만든 함수로 데이터셋 준비
# train dataset
# batch_size = 32

BATCH_SIZE = 32

train_loader, _train_data = create_dataloader(TRAIN_PATH, BATCH_SIZE, True)
target_class_num = len(os.listdir(os.path.join(TRAIN_PATH)))

print('target_class_num: ', target_class_num)
print('train: ', _train_data.class_to_idx)

target_class_num:  4
train:  {'07_inner_cupholder_refined2_train': 0, '08_inner_cupholder_dirt_refined2_train1': 1, '08_inner_cupholder_dirt_refined2_train2': 2, '08_inner_cupholder_dirt_refined2_train3': 3}


In [6]:
# 각 클래스별 이미지 개수 확인
for dirpath, dirnames, filenames in os.walk(TRAIN_PATH):
    print(f'{dirpath} : {len(filenames)}')

/aiffel/aiffel/0_felton/data_ex3_refine/train : 0
/aiffel/aiffel/0_felton/data_ex3_refine/train/08_inner_cupholder_dirt_refined2_train2 : 594
/aiffel/aiffel/0_felton/data_ex3_refine/train/08_inner_cupholder_dirt_refined2_train3 : 732
/aiffel/aiffel/0_felton/data_ex3_refine/train/07_inner_cupholder_refined2_train : 1926
/aiffel/aiffel/0_felton/data_ex3_refine/train/08_inner_cupholder_dirt_refined2_train1 : 600


In [7]:
# test data set 준비
# shuffle = False
test_loader, _test_data = create_dataloader(TEST_PATH, BATCH_SIZE, False)
print('test: ', _test_data.class_to_idx)

test:  {'07_inner_cupholder_refined2_test': 0, '08_inner_cupholder_dirt_refined2_test1': 1, '08_inner_cupholder_dirt_refined2_test2': 2, '08_inner_cupholder_dirt_refined2_test3': 3}


In [8]:
# 개수 확인
for dirpath, dirnames, filenames in os.walk(TEST_PATH):
    print(f'{dirpath} : {len(filenames)}')

/aiffel/aiffel/0_felton/data_ex3_refine/test : 0
/aiffel/aiffel/0_felton/data_ex3_refine/test/08_inner_cupholder_dirt_refined2_test2 : 33
/aiffel/aiffel/0_felton/data_ex3_refine/test/07_inner_cupholder_refined2_test : 100
/aiffel/aiffel/0_felton/data_ex3_refine/test/08_inner_cupholder_dirt_refined2_test3 : 36
/aiffel/aiffel/0_felton/data_ex3_refine/test/08_inner_cupholder_dirt_refined2_test1 : 31


### model 훈련

In [9]:
# Metric 함수
def calculate_metrics(trues, preds):
    accuracy = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='macro')
    precision = precision_score(trues, preds, average='macro')
    recall = recall_score(trues, preds, average='macro')
    return accuracy, f1, precision, recall

In [10]:
# train 함수
def train(dataloader, net, learning_rate, weight_decay_level, device):
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        net.parameters(),
        lr = learning_rate, 
        weight_decay = weight_decay_level
    )

    net.train()

    train_losses = list()
    train_preds = list()
    train_trues = list()
    
    train_loss = 0
    correct = 0
    total = 0

    for idx, (img, label) in enumerate(dataloader):

        img = img.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()

        out = net(img)

        _, pred = torch.max(out, 1)
        loss = criterion(out, label)

        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = out.max(1)
        
        total += label.size(0)
        correct += predicted.eq(label).sum().item() 

        train_losses.append(loss.item())
        train_trues.extend(label.view(-1).cpu().numpy().tolist())
        train_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(train_trues, train_preds)

    print('\n''====== Training Metrics ======')
    print('Loss: ', mean(train_losses))
    print('Acc: ', acc)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', rec)
    print(confusion_matrix(train_trues, train_preds))

    return net, acc, f1, prec, rec

In [11]:
# test 함수
def test(dataloader, net, device):

    criterion = torch.nn.CrossEntropyLoss()
    
    net.eval()
    test_losses = list()
    test_trues = list()
    test_preds = list()
    
    with torch.no_grad():
        for idx, (img, label) in enumerate(dataloader):

            img = img.to(device)
            label = label.to(device)

            out = net(img)

            _, pred = torch.max(out, 1)
            loss = criterion(out, label)

            test_losses.append(loss.item())
            test_trues.extend(label.view(-1).cpu().numpy().tolist())
            test_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(test_trues, test_preds)

    print('====== Test Metrics ======')
    print('Test Loss: ', mean(test_losses))
    print('Test Acc: ', acc)
    print('Test F1: ', f1)
    print('Test Precision: ', prec)
    print('Test Recall: ', rec)
    print(confusion_matrix(test_trues, test_preds))

    return net, acc, f1, prec, rec

In [12]:
# 학습용 함수
# train -> test -> save 반복
# 가장 좋은 accuracy 를 저장

def train_classifier(net, train_loader, test_loader, n_epochs, learning_rate, weight_decay, device):
    best_test_acc = 0
    
    model_save_path = None
    model_save_base = 'weights'
    if not os.path.exists(model_save_base):
        os.makedirs(model_save_base)
    
    print('>> Start Training Model!')
    for epoch in range(n_epochs):
        
        print('> epoch: ', epoch)

        net, _, _, _, _ = train(train_loader, net, learning_rate, weight_decay, device)
        net, test_acc, _, _, _  = test(test_loader, net, device)

        if test_acc > best_test_acc:

            best_test_acc = test_acc
            test_acc_str = '%.5f' % test_acc

            print('[Notification] Best Model Updated!')
            model_save_path = os.path.join(model_save_base, 'classifier_acc_' + str(test_acc_str) + '.pth') 
            torch.save(net.state_dict(), model_save_path)
                
    return model_save_path

In [13]:
# 모델 가져오기
# 사용모델 resNet 50, pretrained = True
# pretraine 된 모델로 전이 학습예정
net = torchvision.models.resnet50(pretrained=True)
net.fc = torch.nn.Linear(
    net.fc.in_features,
    target_class_num
)

net.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /aiffel/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### 모델 학습 시키기

In [14]:
# 1 epoch  시험 
EPOCHS = 1
LEARNING_RATE = 0.005
WEIGHT_DECAY = 0.0005

saved_weight_path = train_classifier(net, train_loader, test_loader, EPOCHS, LEARNING_RATE, WEIGHT_DECAY, device)

>> Start Training Model!
> epoch:  0

====== Training Metrics ======
Loss:  0.6451616164081353
Acc:  0.7198857736240913
F1:  0.5946433273858536
Precision:  0.5935591902009301
Recall:  0.5978630413217514
[[1828   32   26   40]
 [  22  304  185   89]
 [  14  223  189  168]
 [  34  132  114  452]]
====== Test Metrics ======
Test Loss:  1.3871518606320024
Test Acc:  0.43
Test F1:  0.30029625894445044
Test Precision:  0.3641125235404897
Test Recall:  0.37427337895079826
[[57 29  9  5]
 [ 0 26  3  2]
 [ 0 30  2  1]
 [ 0 33  2  1]]
[Notification] Best Model Updated!


In [15]:
# 20 epoch  시험 
EPOCHS = 20
LEARNING_RATE = 0.005
WEIGHT_DECAY = 0.0005

saved_weight_path = train_classifier(net, train_loader, test_loader, EPOCHS, LEARNING_RATE, WEIGHT_DECAY, device)

>> Start Training Model!
> epoch:  0

====== Training Metrics ======
Loss:  0.6206625399510722
Acc:  0.7191069574247144
F1:  0.5846825337944193
Precision:  0.5830955896534346
Recall:  0.5905234524397233
[[1844   13   14   55]
 [  19  302  173  106]
 [  18  245  154  177]
 [  40  111  111  470]]
====== Test Metrics ======
Test Loss:  1.103203152340363
Test Acc:  0.555
Test F1:  0.2806596437074525
Test Precision:  0.307035782124339
Test Recall:  0.33822091886608013
[[100   0   0   0]
 [ 19  10   0   2]
 [ 14  17   1   1]
 [ 18  16   2   0]]
[Notification] Best Model Updated!
> epoch:  1

====== Training Metrics ======
Loss:  0.5346959792385416
Acc:  0.7531152647975078
F1:  0.6149911310089561
Precision:  0.617591294155666
Recall:  0.629679899320048
[[1889    9    8   20]
 [   9  383  116   92]
 [  11  267  127  189]
 [  15  130   85  502]]
====== Test Metrics ======
Test Loss:  0.8619178203654702
Test Acc:  0.635
Test F1:  0.41592534449677304
Test Precision:  0.41933139534883723
Test Reca


====== Training Metrics ======
Loss:  0.41722526998559306
Acc:  0.793613707165109
F1:  0.6668806850876583
Precision:  0.6672426627862744
Recall:  0.6789787106806794
[[1919    1    1    5]
 [   1  385  143   71]
 [   1  312  155  126]
 [   7   48   79  598]]
====== Test Metrics ======
Test Loss:  0.7415274757095024
Test Acc:  0.655
Test F1:  0.47730826084484623
Test Precision:  0.4930091354723708
Test Recall:  0.4875570218312154
[[100   0   0   0]
 [  0  12  11   8]
 [  0  16  14   3]
 [  0  23   8   5]]
> epoch:  17

====== Training Metrics ======
Loss:  0.411605496175033
Acc:  0.79932502596054
F1:  0.6824308099101726
Precision:  0.6819866236126523
Recall:  0.6877968258242504
[[1923    0    0    3]
 [   0  364  172   64]
 [   1  278  202  113]
 [   6   51   85  590]]
====== Test Metrics ======
Test Loss:  1.4925270475542676
Test Acc:  0.485
Test F1:  0.35396920951778743
Test Precision:  0.42339805825242716
Test Recall:  0.3813725969371131
[[67  0  0 33]
 [ 0  1  7 23]
 [ 0  1  7 25]
 